# 🎯 PM OS - Product Manager Operating System

A multi-agent AI assistant for Product Managers.

**Features:**
- 6 specialized agents for different PM tasks
- Automatic intent routing
- Decision log that tracks key recommendations
- Session export for record keeping

**Available Agents:**
- 🔍 **Framer**: Problem definition using 5 Whys
- 📊 **Strategist**: Prioritization with scoring frameworks
- 🤝 **Aligner**: Stakeholder management and meeting prep
- 🚀 **Executor**: MVP scoping and ship checklists
- 📝 **Narrator**: Executive summaries (WHAT/WHY/ASK)
- 📄 **Doc Engine**: PRDs and product documentation

---

## Setup

Run all cells below. You'll need an **Anthropic API key** or **OpenRouter API key**.

In [ ]:
# Step 1: Install dependencies
!pip install -q anthropic gradio

In [ ]:
# Step 2: Define Memory System

import json
from dataclasses import dataclass, field, asdict
from datetime import datetime
from typing import Optional

@dataclass
class Decision:
    timestamp: str
    agent_name: str
    agent_emoji: str
    user_query: str
    decision_summary: str
    context: Optional[str] = None

    def to_dict(self):
        return asdict(self)

@dataclass
class ConversationTurn:
    timestamp: str
    user_message: str
    agent_name: str
    agent_response: str

    def to_dict(self):
        return asdict(self)

@dataclass
class SessionMemory:
    session_id: str
    created_at: str
    conversation: list = field(default_factory=list)
    decisions: list = field(default_factory=list)

    def add_turn(self, user_message, agent_name, agent_response):
        turn = ConversationTurn(
            timestamp=datetime.now().isoformat(),
            user_message=user_message,
            agent_name=agent_name,
            agent_response=agent_response
        )
        self.conversation.append(turn)

    def add_decision(self, agent_name, agent_emoji, user_query, decision_summary, context=None):
        decision = Decision(
            timestamp=datetime.now().isoformat(),
            agent_name=agent_name,
            agent_emoji=agent_emoji,
            user_query=user_query,
            decision_summary=decision_summary,
            context=context
        )
        self.decisions.append(decision)

    def get_decisions_markdown(self):
        if not self.decisions:
            return "*No decisions logged yet. Start chatting to build your decision log!*"
        lines = ["## 📋 Decision Log\n"]
        for i, d in enumerate(self.decisions, 1):
            time_str = datetime.fromisoformat(d.timestamp).strftime("%H:%M")
            lines.append(f"### {i}. {d.agent_emoji} {d.agent_name} ({time_str})")
            lines.append(f"**Query:** {d.user_query[:100]}{'...' if len(d.user_query) > 100 else ''}")
            lines.append(f"**Decision:** {d.decision_summary}")
            if d.context:
                lines.append(f"*Context: {d.context}*")
            lines.append("")
        return "\n".join(lines)

    def to_dict(self):
        return {
            "session_id": self.session_id,
            "created_at": self.created_at,
            "conversation": [t.to_dict() for t in self.conversation],
            "decisions": [d.to_dict() for d in self.decisions]
        }

    def save(self, filepath=None):
        if filepath is None:
            filepath = f"pm_os_session_{self.session_id}.json"
        with open(filepath, "w") as f:
            json.dump(self.to_dict(), f, indent=2)
        return filepath

def create_session():
    session_id = datetime.now().strftime("%Y%m%d_%H%M%S")
    return SessionMemory(session_id=session_id, created_at=datetime.now().isoformat())

def extract_decision_summary(agent_name, response):
    response_lower = response.lower()
    if agent_name == "strategist" and "**recommendation:**" in response_lower:
        start = response_lower.find("**recommendation:**")
        end = response.find("\n\n", start)
        if end == -1: end = start + 200
        return response[start:end].replace("**Recommendation:**", "").strip()[:200]
    elif agent_name == "framer" and "**problem statement:**" in response_lower:
        start = response_lower.find("**problem statement:**")
        end = response.find("\n\n", start)
        if end == -1: end = start + 200
        return response[start:end].replace("**Problem Statement:**", "").strip()[:200]
    elif agent_name == "executor" and ("**mvp definition:**" in response_lower or "the mvp includes only:" in response_lower):
        start = response_lower.find("mvp")
        end = response.find("\n\n", start + 50)
        if end == -1: end = start + 200
        return "MVP defined: " + response[start:end].strip()[:150]
    elif agent_name == "aligner" and "**the ask:**" in response_lower:
        start = response_lower.find("**the ask:**")
        end = response.find("\n\n", start)
        if end == -1: end = start + 200
        return response[start:end].replace("**The Ask:**", "").strip()[:200]
    elif agent_name == "narrator" and "**tl;dr:**" in response_lower:
        start = response_lower.find("**tl;dr:**")
        end = response.find("\n", start + 10)
        if end == -1: end = start + 200
        return response[start:end].replace("**TL;DR:**", "").strip()[:200]
    elif agent_name == "doc_engine" and "**product name:**" in response_lower:
        start = response_lower.find("**product name:**")
        end = response.find("\n", start)
        return "Document created: " + response[start:end].replace("**Product Name:**", "").strip()[:100]
    # Fallback
    lines = response.split("\n")
    for line in lines:
        line = line.strip()
        if line and not line.startswith("#") and not line.startswith("*") and len(line) > 20:
            return line[:200]
    return None

print("✅ Memory system loaded!")

In [ ]:
# Step 3: Define Agents

from dataclasses import dataclass
import anthropic

@dataclass
class Agent:
    name: str
    emoji: str
    description: str
    system_prompt: str

    @property
    def display_name(self):
        return f"{self.emoji} {self.name} Agent"

FRAMER = Agent(
    name="Framer",
    emoji="🔍",
    description="For vague problems - uses 5 Whys to find root cause",
    system_prompt="""You are the Framer Agent, a PM expert at problem definition.

Your role: Take vague, unclear problems and help define them precisely using the 5 Whys technique.

OUTPUT FORMAT:
## Problem Analysis

**Surface Problem:** [What the user described]

**5 Whys Deep Dive:**
1. Why? → [First level answer]
2. Why? → [Second level answer]
3. Why? → [Third level answer]
4. Why? → [Fourth level answer]
5. Why? → [Root cause]

**Root Cause Identified:** [Clear statement]

**Problem Statement:**
> [One clear, actionable problem statement in format: "[User/Customer] needs [need] because [insight]"]

**Recommended Next Steps:**
- [Action 1]
- [Action 2]
- [Action 3]"""
)

STRATEGIST = Agent(
    name="Strategist",
    emoji="📊",
    description="For prioritization decisions - creates scoring frameworks",
    system_prompt="""You are the Strategist Agent, a PM expert at prioritization.

Your role: Help PMs make clear prioritization decisions using structured frameworks.

OUTPUT FORMAT:
## Prioritization Analysis

**Options Under Consideration:**
1. [Option A]
2. [Option B]

**Scoring Criteria:**
- Impact (1-5): Business/user value delivered
- Effort (1-5): Resources and time required (lower = easier)
- Strategic Fit (1-5): Alignment with company goals
- Risk (1-5): Confidence in execution (higher = lower risk)

**Scoring Matrix:**

| Option | Impact | Effort | Strategic Fit | Risk | Total |
|--------|--------|--------|---------------|------|-------|
| [A]    | X      | X      | X             | X    | XX    |
| [B]    | X      | X      | X             | X    | XX    |

**Recommendation:** [Clear choice with reasoning]

**Key Considerations:**
- [Trade-off 1]
- [Trade-off 2]

**Next Steps:**
- [Action 1]
- [Action 2]

Be decisive. PMs need clear recommendations, not just frameworks."""
)

ALIGNER = Agent(
    name="Aligner",
    emoji="🤝",
    description="For stakeholder management - maps motivations and preps talking points",
    system_prompt="""You are the Aligner Agent, a PM expert at stakeholder management.

Your role: Help PMs navigate stakeholder dynamics and prepare for alignment conversations.

OUTPUT FORMAT:
## Stakeholder Alignment Plan

**Context:** [Brief situation summary]

**Stakeholder Map:**

### [Stakeholder 1: Role/Name]
- **Motivations:** What they care about
- **Concerns:** What worries them
- **Success Metrics:** How they're measured
- **Your Ask:** What you need from them
- **Their Win:** How this helps them

**Talking Points:**
1. **Opening:** [How to frame the conversation]
2. **Key Points:**
   - [Point 1 with supporting data]
   - [Point 2 with supporting data]
3. **Anticipated Objections:**
   - "[Objection]" → [Your response]
4. **The Ask:** [Clear, specific request]

**Pre-Meeting Checklist:**
- [ ] [Prep item 1]
- [ ] [Prep item 2]"""
)

EXECUTOR = Agent(
    name="Executor",
    emoji="🚀",
    description="For shipping - cuts scope to MVP and creates action checklists",
    system_prompt="""You are the Executor Agent, a PM expert at shipping products.

Your role: Help PMs cut scope ruthlessly, define true MVPs, and create actionable checklists.

OUTPUT FORMAT:
## Execution Plan

**Goal:** [What we're shipping and why]

**Core Value Proposition:** [The ONE thing this must do well]

**Scope Analysis:**

| Feature | Must Have | Nice to Have | Cut |
|---------|-----------|--------------|-----|
| [Feature 1] | ✅ | | |
| [Feature 2] | | ✅ | |
| [Feature 3] | | | ❌ |

**MVP Definition:**
The MVP includes ONLY:
1. [Essential feature 1]
2. [Essential feature 2]
3. [Essential feature 3]

**What We're NOT Building (v1):**
- ~~[Cut item 1]~~ - Why: [reason]
- ~~[Cut item 2]~~ - Why: [reason]

**Ship Checklist:**
- [ ] [Task 1] - Owner: [who]
- [ ] [Task 2] - Owner: [who]

Be ruthless about scope. The goal is to SHIP."""
)

NARRATOR = Agent(
    name="Narrator",
    emoji="📝",
    description="For communication - writes exec summaries in WHAT/WHY/ASK format",
    system_prompt="""You are the Narrator Agent, a PM expert at executive communication.

Your role: Help PMs communicate clearly using the WHAT/WHY/ASK framework.

OUTPUT FORMAT:
## Executive Summary

**TL;DR:** [One sentence summary - the bottom line upfront]

---

### WHAT
[2-3 sentences max. What is happening/what are we doing? Facts only.]

### WHY
[2-3 sentences max. Why does this matter? Business impact, urgency, opportunity cost.]

### ASK
[Specific, clear request. What do you need from the reader?]
- **Decision needed:** [Yes/No + what decision]
- **By when:** [Date/urgency]
- **From whom:** [Specific person/group]

---

**Supporting Context:** (if needed)
- [Key data point 1]
- [Key data point 2]

Be concise. Executives have 30 seconds. Make every word count."""
)

DOC_ENGINE = Agent(
    name="Doc Engine",
    emoji="📄",
    description="For documents - generates PRDs, specs, and other PM artifacts",
    system_prompt="""You are the Doc Engine Agent, a PM expert at creating product documentation.

Your role: Generate high-quality PRDs, specs, and other PM artifacts quickly.

OUTPUT FORMAT FOR PRD:
## Product Requirements Document

### Overview
**Product Name:** [Name]
**Author:** [PM Name]
**Last Updated:** [Date]
**Status:** Draft

### Problem Statement
[What problem are we solving? For whom?]

### Goals & Success Metrics
**Primary Goal:** [One clear goal]

| Metric | Current | Target | Timeline |
|--------|---------|--------|----------|
| [Metric 1] | X | Y | [Date] |

### User Stories
1. As a [user], I want to [action] so that [benefit]

### Requirements
| ID | Requirement | Priority |
|----|-------------|----------|
| FR1 | [Requirement] | P0/P1/P2 |

### Scope
**In Scope:** [Items]
**Out of Scope:** [Items]

### Timeline
| Phase | Deliverable | Date |
|-------|-------------|------|

### Open Questions
- [ ] [Question 1]"""
)

AGENTS = {
    "framer": FRAMER,
    "strategist": STRATEGIST,
    "aligner": ALIGNER,
    "executor": EXECUTOR,
    "narrator": NARRATOR,
    "doc_engine": DOC_ENGINE,
}

def get_client(api_key, provider="anthropic"):
    if provider == "openrouter":
        return anthropic.Anthropic(api_key=api_key, base_url="https://openrouter.ai/api/v1")
    return anthropic.Anthropic(api_key=api_key)

def get_model(provider="anthropic"):
    if provider == "openrouter":
        return "anthropic/claude-sonnet-4"
    return "claude-sonnet-4-20250514"

def generate_response(agent, user_message, conversation_history, api_key, provider="anthropic"):
    client = get_client(api_key, provider)
    model = get_model(provider)
    messages = [{"role": msg["role"], "content": msg["content"]} for msg in conversation_history]
    messages.append({"role": "user", "content": user_message})
    response = client.messages.create(model=model, max_tokens=2048, system=agent.system_prompt, messages=messages)
    return response.content[0].text

print("✅ Agents loaded!")
for name, agent in AGENTS.items():
    print(f"   {agent.display_name}")

In [ ]:
# Step 4: Define Router

ROUTER_SYSTEM_PROMPT = """You are an intent classifier for a PM assistant system.

Analyze the user message and return ONLY the agent name (lowercase, no explanation):

1. **framer** - Vague problems, "users are doing X but not Y", symptoms without clear causes
2. **strategist** - "Should we X or Y?", prioritization, trade-offs, roadmap decisions
3. **aligner** - Stakeholder prep, meetings with executives, getting buy-in, politics
4. **executor** - Shipping, MVP scoping, cutting features, launch checklists
5. **narrator** - Exec summaries, status updates, communicating to leadership
6. **doc_engine** - "Write a PRD", "create a spec", formal documentation

Respond with ONLY the agent name. Examples:
- "Users sign up but don't complete onboarding" → framer
- "Should we build AI or security?" → strategist
- "Meeting with CEO tomorrow" → aligner
- "Cut this to MVP" → executor
- "Write exec summary" → narrator
- "Write a PRD for onboarding" → doc_engine
"""

def classify_intent(user_message, api_key, provider="anthropic"):
    client = get_client(api_key, provider)
    model = get_model(provider)
    response = client.messages.create(
        model=model,
        max_tokens=50,
        system=ROUTER_SYSTEM_PROMPT,
        messages=[{"role": "user", "content": user_message}]
    )
    agent_name = response.content[0].text.strip().lower()
    if agent_name not in AGENTS:
        return "framer"
    return agent_name

def route_message(user_message, api_key, provider="anthropic"):
    agent_name = classify_intent(user_message, api_key, provider)
    return agent_name, AGENTS[agent_name]

print("✅ Router loaded!")

In [ ]:
# Step 5: Launch the Gradio App

import gradio as gr

# Initialize session memory
session_memory = create_session()

def format_agent_header(agent):
    return f"### {agent.display_name}\n*{agent.description}*\n\n---\n\n"

def chat(message, history, api_key, provider):
    global session_memory

    if not message.strip():
        return history, "", session_memory.get_decisions_markdown()
    if not api_key.strip():
        history.append([message, "⚠️ Please enter your API key above."])
        return history, "", session_memory.get_decisions_markdown()

    provider_key = "openrouter" if provider == "OpenRouter" else "anthropic"

    try:
        agent_name, agent = route_message(message, api_key, provider_key)
        conversation_history = []
        for user_msg, assistant_msg in history:
            conversation_history.append({"role": "user", "content": user_msg})
            clean_response = assistant_msg
            if assistant_msg.startswith("###"):
                divider_pos = assistant_msg.find("---\n\n")
                if divider_pos != -1:
                    clean_response = assistant_msg[divider_pos + 5:]
            conversation_history.append({"role": "assistant", "content": clean_response})

        response = generate_response(agent, message, conversation_history, api_key, provider_key)
        formatted_response = format_agent_header(agent) + response
        history.append([message, formatted_response])

        # Log to memory
        session_memory.add_turn(message, agent_name, response)

        # Extract and log decision
        decision_summary = extract_decision_summary(agent_name, response)
        if decision_summary:
            session_memory.add_decision(
                agent_name=agent.name,
                agent_emoji=agent.emoji,
                user_query=message,
                decision_summary=decision_summary
            )

    except Exception as e:
        history.append([message, f"**Error:** {str(e)}"])

    return history, "", session_memory.get_decisions_markdown()

def clear_chat():
    global session_memory
    session_memory = create_session()
    return [], "", session_memory.get_decisions_markdown()

def export_session():
    global session_memory
    if not session_memory.decisions:
        return "No decisions to export yet."
    filepath = session_memory.save()
    return f"Session exported to `{filepath}`"

with gr.Blocks(title="PM OS") as app:
    gr.Markdown("""
    # 🎯 PM OS
    ### Product Manager Operating System

    A multi-agent AI assistant for PMs. Describe what you need, and the right agent is selected automatically.
    """)

    with gr.Row():
        provider_select = gr.Dropdown(label="Provider", choices=["Anthropic", "OpenRouter"], value="Anthropic", scale=1)
        api_key_input = gr.Textbox(label="API Key", placeholder="sk-ant-... or sk-or-...", type="password", scale=3)

    with gr.Tabs():
        with gr.TabItem("💬 Chat"):
            chatbot = gr.Chatbot(label="Chat", height=400)
            with gr.Row():
                msg_input = gr.Textbox(placeholder="e.g., 'Should we prioritize AI features or enterprise security?'", scale=4, show_label=False)
                submit_btn = gr.Button("Send", variant="primary", scale=1)
            clear_btn = gr.Button("Clear Chat")

            gr.Markdown("**Try these examples:**")
            gr.Examples(
                examples=[
                    "Should we prioritize AI features or enterprise security?",
                    "Users are signing up but not completing onboarding",
                    "I have a meeting with my CEO tomorrow about Q1 priorities",
                    "Write a PRD for a new onboarding flow",
                    "Help me cut this feature list to an MVP",
                ],
                inputs=msg_input
            )

        with gr.TabItem("📋 Decision Log"):
            gr.Markdown("*Decisions are automatically logged here as you chat.*")
            decision_log = gr.Markdown(value="*No decisions logged yet.*")
            with gr.Row():
                export_btn = gr.Button("Export Session")
                export_status = gr.Textbox(show_label=False, interactive=False)

        with gr.TabItem("ℹ️ Agents"):
            gr.Markdown("""
            ## Available Agents

            | Agent | Trigger | Output |
            |-------|---------|--------|
            | 🔍 **Framer** | Vague problems | 5 Whys → Problem Statement |
            | 📊 **Strategist** | "Should we X or Y?" | Scoring Matrix → Recommendation |
            | 🤝 **Aligner** | Meetings, stakeholders | Stakeholder Map → Talking Points |
            | 🚀 **Executor** | "Ship", "MVP" | Scope Analysis → Checklist |
            | 📝 **Narrator** | "Summarize" | WHAT/WHY/ASK Summary |
            | 📄 **Doc Engine** | "Write a PRD" | Full Document |
            """)

    submit_btn.click(fn=chat, inputs=[msg_input, chatbot, api_key_input, provider_select], outputs=[chatbot, msg_input, decision_log])
    msg_input.submit(fn=chat, inputs=[msg_input, chatbot, api_key_input, provider_select], outputs=[chatbot, msg_input, decision_log])
    clear_btn.click(fn=clear_chat, outputs=[chatbot, msg_input, decision_log])
    export_btn.click(fn=export_session, outputs=[export_status])

print("🚀 Launching PM OS...")
app.launch(share=True, debug=True)